In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


#MNIST

In [ ]:
# Create the teacher
tf.random.set_seed(1234)

teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(100),
        layers.Dense(10),
    ],
    name="teacher",
)

tf.random.set_seed(1234)

teacher_asst = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(50),
        layers.Dense(10),
    ],
    name="teacher",
)

tf.random.set_seed(1234)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 14, 14, 128)       1280      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 256)         295168    
                                                                 
 flatten_3 (Flatten)         (None, 12544)             0         
                                                                 
 dense_5 (Dense)             (None, 100)               1254500   
                                                           

In [ ]:
teacher_asst.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten_4 (Flatten)         (None, 3136)              0         
                                                                 
 dense_7 (Dense)             (None, 50)                156850    
                                                           

In [ ]:
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 14, 14, 16)        160       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 7, 7, 32)          4640      
                                                                 
 flatten_5 (Flatten)         (None, 1568)              0         
                                                                 
 dense_9 (Dense)             (None, 10)                15690     
                                                           

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 24s 8ms/step - loss: 0.1471 - sparse_categorical_accuracy: 0.9553
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0887 - sparse_categorical_accuracy: 0.9731
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0759 - sparse_categorical_accuracy: 0.9770
Epoch 4/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0687 - sparse_categorical_accuracy: 0.9792
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.1013 - sparse_categorical_accuracy: 0.9700


[0.10134230554103851, 0.9700000286102295]

In [ ]:
tf.random.set_seed(1234)

teacher_asst = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(50),
        layers.Dense(10),
    ],
    name="teacher",
)

tf.random.set_seed(1234)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=teacher_asst, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 14s 7ms/step - sparse_categorical_accuracy: 0.9500 - student_loss: 0.1650 - distillation_loss: 0.0042
Epoch 2/3
1875/1875 [==============================] - 14s 7ms/step - sparse_categorical_accuracy: 0.9769 - student_loss: 0.0775 - distillation_loss: 0.0022
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9798 - student_loss: 0.0694


[0.9797999858856201, 0.0026290188543498516]

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=distiller.student)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 10s 5ms/step - sparse_categorical_accuracy: 0.9259 - student_loss: 0.2447 - distillation_loss: 0.0056
Epoch 2/3
1875/1875 [==============================] - 9s 5ms/step - sparse_categorical_accuracy: 0.9694 - student_loss: 0.1018 - distillation_loss: 0.0021
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9758 - student_loss: 0.0797


[0.9757999777793884, 0.005773582495748997]

In [ ]:
tf.random.set_seed(1234)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 13s 7ms/step - sparse_categorical_accuracy: 0.9262 - student_loss: 0.2438 - distillation_loss: 0.0064
Epoch 2/3
1875/1875 [==============================] - 12s 7ms/step - sparse_categorical_accuracy: 0.9698 - student_loss: 0.1013 - distillation_loss: 0.0031
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9768 - student_loss: 0.0734


[0.9768000245094299, 0.004563521593809128]

In [ ]:
# Train student
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2305 - sparse_categorical_accuracy: 0.9307
Epoch 2/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0925 - sparse_categorical_accuracy: 0.9723
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.0774 - sparse_categorical_accuracy: 0.9755


[0.07737163454294205, 0.9754999876022339]

#CIFAR 10

In [ ]:
# Prepare the train and test dataset.
batch_size = 128
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 32, 32, 3))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 32, 32, 3))


170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
# Create the teacher
tf.random.set_seed(1234)
teacher = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(256),
        layers.Dropout(0.5),
        layers.Dense(256),
        layers.Dense(10),
    ],
    name="teacher",
)

tf.random.set_seed(1234)

teacher_asst = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
tf.random.set_seed(1234)
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=10)
teacher.evaluate(x_test, y_test)

Epoch 1/10
1563/1563 [==============================] - 22s 14ms/step - loss: 1.4635 - sparse_categorical_accuracy: 0.4707
Epoch 2/10
1563/1563 [==============================] - 21s 14ms/step - loss: 1.0665 - sparse_categorical_accuracy: 0.6291
Epoch 3/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.9037 - sparse_categorical_accuracy: 0.6872
Epoch 4/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.7861 - sparse_categorical_accuracy: 0.7304
Epoch 5/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.6892 - sparse_categorical_accuracy: 0.7611
Epoch 6/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.6023 - sparse_categorical_accuracy: 0.7933
Epoch 7/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.5365 - sparse_categorical_accuracy: 0.8157
Epoch 8/10
1563/1563 [==============================] - 21s 14ms/step - loss: 0.4690 - sparse_categorical_accuracy: 0.8417
Epoch 9/10
1563/

[1.2077758312225342, 0.6980000138282776]

In [ ]:
tf.random.set_seed(1234)

teacher_asst = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
tf.random.set_seed(1234)
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=teacher_asst, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 26s 16ms/step - sparse_categorical_accuracy: 0.4814 - student_loss: 1.4389 - distillation_loss: 0.0094
Epoch 2/5
1563/1563 [==============================] - 24s 16ms/step - sparse_categorical_accuracy: 0.6428 - student_loss: 1.0356 - distillation_loss: 0.0066
Epoch 3/5
1563/1563 [==============================] - 24s 16ms/step - sparse_categorical_accuracy: 0.7009 - student_loss: 0.8738 - distillation_loss: 0.0055
Epoch 4/5
1563/1563 [==============================] - 25s 16ms/step - sparse_categorical_accuracy: 0.7465 - student_loss: 0.7407 - distillation_loss: 0.0048
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - sparse_categorical_accuracy: 0.7040 - student_loss: 0.9190


[0.7039999961853027, 1.1463186740875244]

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=distiller.student)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 19s 12ms/step - sparse_categorical_accuracy: 0.4704 - student_loss: 1.4605 - distillation_loss: 0.0042
Epoch 2/5
1563/1563 [==============================] - 18s 11ms/step - sparse_categorical_accuracy: 0.6291 - student_loss: 1.0660 - distillation_loss: 0.0027
Epoch 3/5
1563/1563 [==============================] - 17s 11ms/step - sparse_categorical_accuracy: 0.6872 - student_loss: 0.9066 - distillation_loss: 0.0021
Epoch 4/5
1563/1563 [==============================] - 17s 11ms/step - sparse_categorical_accuracy: 0.7218 - student_loss: 0.7948 - distillation_loss: 0.0019
Epoch 5/5
313/313 [==============================] - 2s 4ms/step - sparse_categorical_accuracy: 0.7027 - student_loss: 0.8815


[0.7027000188827515, 1.6460639238357544]

In [ ]:
# Create the student
tf.random.set_seed(1234)
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 19s 11ms/step - sparse_categorical_accuracy: 0.4712 - student_loss: 1.4595 - distillation_loss: 0.0096
Epoch 2/5
1563/1563 [==============================] - 17s 11ms/step - sparse_categorical_accuracy: 0.6246 - student_loss: 1.0737 - distillation_loss: 0.0071
Epoch 3/5
1563/1563 [==============================] - 17s 11ms/step - sparse_categorical_accuracy: 0.6809 - student_loss: 0.9154 - distillation_loss: 0.0060
Epoch 4/5
1563/1563 [==============================] - 17s 11ms/step - sparse_categorical_accuracy: 0.7218 - student_loss: 0.8023 - distillation_loss: 0.0053
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.6846 - student_loss: 0.9451


[0.6845999956130981, 1.4397974014282227]

In [ ]:
# Train student 
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=5)
student_scratch.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 15s 9ms/step - loss: 1.0938 - sparse_categorical_accuracy: 0.6135
Epoch 2/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.9330 - sparse_categorical_accuracy: 0.6764
Epoch 3/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.8260 - sparse_categorical_accuracy: 0.7127
Epoch 4/5
1563/1563 [==============================] - 14s 9ms/step - loss: 0.7254 - sparse_categorical_accuracy: 0.7460
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.8921 - sparse_categorical_accuracy: 0.7039


[0.8920611143112183, 0.7038999795913696]

In [ ]:
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.5),
        layers.Dense(128),
        layers.Dense(10),
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=2,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=5)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 23s 14ms/step - sparse_categorical_accuracy: 0.4760 - student_loss: 1.8499 - distillation_loss: 0.0196
Epoch 2/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.6404 - student_loss: 1.6112 - distillation_loss: 0.0287
Epoch 3/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.6983 - student_loss: 1.5246 - distillation_loss: 0.0318
Epoch 4/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.7353 - student_loss: 1.4635 - distillation_loss: 0.0339
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.7145 - student_loss: 1.4489


[0.7145000100135803, 1.4703700542449951]

In [ ]:
for T in [5,10,20,30,40]:
  student = keras.Sequential(
      [
          keras.Input(shape=(32, 32, 3)),
          layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
          layers.Flatten(),
          layers.Dense(128),
          layers.Dropout(0.5),
          layers.Dense(128),
          layers.Dense(10),
      ],
      name="student",
  )

  # Initialize and compile distiller
  distiller = Distiller(student=student, teacher=teacher)
  distiller.compile(
      optimizer=keras.optimizers.Adam(),
      metrics=[keras.metrics.SparseCategoricalAccuracy()],
      student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      distillation_loss_fn=keras.losses.KLDivergence(),
      alpha=0.1,
      temperature=T,
  )

  # Distill teacher to student
  distiller.fit(x_train, y_train, epochs=5)

  # Evaluate student on test dataset
  print('Temp = ', T)
  print(distiller.evaluate(x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 23s 14ms/step - sparse_categorical_accuracy: 0.4825 - student_loss: 1.5346 - distillation_loss: 0.0174
Epoch 2/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.6448 - student_loss: 1.1765 - distillation_loss: 0.0239
Epoch 3/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.7020 - student_loss: 1.0396 - distillation_loss: 0.0261
Epoch 4/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.7433 - student_loss: 0.9455 - distillation_loss: 0.0278
Epoch 5/5
1563/1563 [==============================] - 22s 14ms/step - sparse_categorical_accuracy: 0.7718 - student_loss: 0.8760 - distillation_loss: 0.0290
Temp =  5
313/313 [==============================] - 2s 5ms/step - sparse_categorical_accuracy: 0.7080 - student_loss: 0.9809
[0.7080000042915344, 0.9686511754989624]
Epoch 1/5
1563/1563 [======================

#Testing old model for CIFAR10


In [ ]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Flatten(),
        layers.Dense(256),
        layers.Dropout(0.5),
        layers.Dense(256),
        layers.Dropout(0.5),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=20,validation_split=0.1)
teacher.evaluate(x_test, y_test)

Epoch 1/20
1407/1407 [==============================] - 17s 12ms/step - loss: 1.5213 - sparse_categorical_accuracy: 0.4454 - val_loss: 1.2452 - val_sparse_categorical_accuracy: 0.5482
Epoch 2/20
1407/1407 [==============================] - 15s 11ms/step - loss: 1.1253 - sparse_categorical_accuracy: 0.6080 - val_loss: 0.9894 - val_sparse_categorical_accuracy: 0.6496
Epoch 3/20
1407/1407 [==============================] - 15s 11ms/step - loss: 0.9659 - sparse_categorical_accuracy: 0.6662 - val_loss: 0.9021 - val_sparse_categorical_accuracy: 0.6890
Epoch 4/20
1407/1407 [==============================] - 15s 11ms/step - loss: 0.8630 - sparse_categorical_accuracy: 0.7026 - val_loss: 0.9233 - val_sparse_categorical_accuracy: 0.6802
Epoch 5/20
1407/1407 [==============================] - 14s 10ms/step - loss: 0.7819 - sparse_categorical_accuracy: 0.7297 - val_loss: 0.8963 - val_sparse_categorical_accuracy: 0.7028
Epoch 6/20
1407/1407 [==============================] - 15s 11ms/step - loss: 0.

[1.6273808479309082, 0.6682000160217285]